In [1]:
from key import key

In [2]:
!pip install openai

In [3]:
import os
import openai
from openai import OpenAI
openai.api_key = key
client=OpenAI(api_key=key)
#client = openai.ChatCompletion()



In [4]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Say this is a test"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='This is a test', role='assistant', function_call=None, tool_calls=None)


In [5]:
!pip install pdfplumber

In [6]:
import pdfplumber
import requests
from bs4 import BeautifulSoup

def extract_text_from_pdfs(pdf_paths):
    all_text = ""
    for pdf_path in pdf_paths:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"
    return all_text

def extract_text_from_urls(urls):
    all_text = ""
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_text += soup.get_text() + "\n"
    return all_text


In [7]:
pdf_paths = [
    '/content/ATTACK_Design_and_Philosophy_March_2020.pdf',
    '/content/CELEX_32016R0679_EN_TXT.pdf',
    '/content/diamond.pdf',
    '/content/nist.sp.800-150.pdf'
]
urls = [
    'https://oasis-open.github.io/cti-documentation/',
    'https://capec.mitre.org'
]

pdf_text = extract_text_from_pdfs(pdf_paths)
url_text = extract_text_from_urls(urls)
combined_text = pdf_text + url_text
text_chunks = [combined_text[i:i + 2000] for i in range(0, len(combined_text), 2000)]  # Adjust chunk size as needed

In [8]:
def generate_question(chunk, max_tokens=150):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a cybersecurity expert specializing in cyber threat intelligence."},
                {"role": "user", "content": f"Given the text below, please generate a single multiple-choice question with four options based on the following requirements:\n"
                                            "1. Question Format: The question must have four options. The options should be challenging and require careful consideration. Avoid creating options that could be interpreted as correct under different circumstances.\n"
                                            "2. Target Audience: The question should be suitable for security professionals with three to five years of experience in cyber threat intelligence. Avoid generic questions such as “What is the objective?”, “Which operating system can be targeted?”.\n"
                                            "3. Content Coverage: Aim to cover various sections of the document to ensure a comprehensive evaluation of the candidate’s knowledge. Include context-specific questions that require an understanding of the document’s content.\n"
                                            "4. Technical Precision: Use precise terminology and concepts relevant to cyber threat intelligence. Incorporate situational or scenario-based questions where applicable.\n"
                                            "5. Include Technique IDs and Names: Ensure that the question, where applicable, mentions both the ID and the full name of the MITRE ATT&CK pattern technique.\n"
                                            "6. Premise Inclusion: The question should include a premise indicating it pertains to MITRE ATT&CK, specifying the relevant platform (Enterprise, ICS, or Mobile) where necessary.\n"
                                            "7. Output Format: Return the output in TSV format (must be tab-separated) with the following columns: Question, Option A, Option B, Option C, Option D, Correct Answer (A, B, C, D), and Explanation.\n"
                                            "Important: Only return the TSV content as specified. Do not include any additional text or commentary outside the TSV format.\n"
                                            "Text:\n\n{chunk}"}
            ],
            max_tokens=max_tokens,
            temperature=0.7
        )

        if response:
            tsv_content = response.choices[0].message.content.strip()
            # Remove the header row from the TSV string
            lines = tsv_content.split('\n')
            if len(lines) > 1:
                return '\n'.join(lines[1:])
            else:
                return tsv_content
        else:
            return None
    except Exception as e:
        print(f"Error generating question: {e}")
        return ""

In [15]:
def process_and_generate_question(pdf_paths, urls):
    questions = []
    for i in range(2500):
        # Generate a chunk of text for each question
        chunk = combined_text[i*2000:(i+1)*2000]  # Use a 2000-character chunk for each question
        tsv_content = generate_question(chunk)
        if tsv_content:
            questions.append(tsv_content)
        else:
            print(f"Failed to generate question {i+1}.")
    return questions

# Example usage:
questions = process_and_generate_question(pdf_paths, urls)
if questions:
    print("Question\tAnswer\tReasoning Skill")  # Print the header once
    for question in questions:
        print(question)
else:
    print("Failed to generate any questions.")

Question	Answer	Reasoning Skill
Which MITRE ATT&CK technique involves an adversary directly interacting with a victim to manipulate them into divulging confidential information?	Spearphishing Attachment (T1193)	Social Engineering (T1140)	Remote Access Tools (T1219)	Man-in-the-Middle Attack (T1557)	B	Social Engineering (T1140) is the correct answer as it specifically involves manipulating individuals into performing actions or divulging confidential information. Spearphishing Attachment (T1193) focuses on sending malicious attachments via email. Remote Access Tools (T1219) refers to tools allowing remote access to systems. Man-in-the-Middle Attack
Which MITRE ATT&CK technique involves an adversary exploiting a weakness in a third-party service that is used in the target environment to gain initial access?	(A) Exploit Public-Facing Application (T1190)	(B) Third-party Software (T1072)	(C) Exploitation for Client Execution (T1203)	(D) Exploitation for Privilege Escalation (T1068)	B	Adversa

In [20]:
import csv

def save_questions_to_tsv(questions, output_file):
    header = ["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Explanation"]

    with open(output_file, 'w', newline='') as tsvfile:
        writer = csv.writer(tsvfile, delimiter='\t')
        writer.writerow(header)
        for question in questions:
            lines = question.split('\n')
            for line in lines:
                # Split the line by tabs and handle potential empty fields
                fields = line.split('\t')
                # Ensure that there are exactly 7 fields by padding or truncating as necessary
                if len(fields) < 7:
                    fields += [''] * (7 - len(fields))  # Pad with empty strings
                elif len(fields) > 7:
                    fields = fields[:7]  # Truncate to 7 fields
                writer.writerow(fields)

save_questions_to_tsv(questions, 'output_questions.tsv')

In [23]:
import pandas as pd
df = pd.read_csv('/content/output_questions.tsv', sep='\t')
df

,Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation
0,Which MITRE ATT&CK technique involves an adver...,Spearphishing Attachment (T1193),Social Engineering (T1140),Remote Access Tools (T1219),Man-in-the-Middle Attack (T1557),B,Social Engineering (T1140) is the correct answ...
1,Which MITRE ATT&CK technique involves an adver...,(A) Exploit Public-Facing Application (T1190),(B) Third-party Software (T1072),(C) Exploitation for Client Execution (T1203),(D) Exploitation for Privilege Escalation (T1068),B,Adversaries can exploit weaknesses in third-pa...
2,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Data Staged (T1074),Command and Control (T1009),Remote File Copy (T1105),C,Command and Control (T1009) corresponds to adv...
3,"In the context of MITRE ATT&CK for Enterprise,...",T1548.002 - Abuse Elevation Control Mechanism,T1059.001 - Command-Line Interface,T1566.001 - Phishing,T1078.002 - Valid Accounts,T1548.002 - Abuse Elevation Control Mechanism,Adversaries may abuse elevation control mechan...
4,Which MITRE ATT&CK technique involves an adver...,Process Injection (T1055) - Process Injection,Man-in-the-Middle (T1557) - Man-in-the-Middle,Control Device Identification (T818) - Control...,Manipulate Device Communication (T848) - Manip...,C,Manipulate Device Communication (T848) involve...
...,...,...,...,...,...,...,...
2513,Which MITRE ATT&CK technique involves an adver...,Process Injection - T1055,Defense Evasion - T1562,Execution Guardrails - T1480,Exploit Public-Facing Application - T1190,A,Process Injection is the technique where adver...
2514,Based on the provided document related to Cybe...,SYSTEM SERVICE DISCOVERY,ARCHIVE COLLECTED DATA,BYPASS USER ACCOUNT CONTROL,INPUT CAPTURE,B,Technique T1560 - Archive Collected Data invol...
2515,"In the context of MITRE ATT&CK for Enterprise,...",Process Discovery (T1057) - Process Discovery,Data Staging (T1074) - Data Staging,Data from Information Repositories (T1213) - D...,Remote System Discovery (T1018) - Remote Syste...,Remote System Discovery (T1018) - Remote Syste...,The technique described involves the use of ex...
2516,"In the context of MITRE ATT&CK techniques, whi...",Execution,Exfiltration,Collection,Lateral Movement,C,Adversaries using tools to collect and exfiltr...


In [12]:
# prompt: Using dataframe df: slipt the colomn and make content under

import pandas as pd

# Assuming 'df' is your DataFrame with a single column containing tab-separated data
new_df = df[0].str.split('\t', expand=True)

# Set the column names
new_df.columns = ["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Explanation"]

# Display the first few rows of the new DataFrame
new_df.head()


,Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation
0,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection - Dynamic-link Library Injec...,Valid Accounts (T1078),Remote File Copy (T1105),New Service (T1050),B,Adversaries may create a new account on a syst...
1,Which MITRE ATT&CK technique involves an adver...,Network Sniffing (T1040),Masquerading (T1036),Connection Proxy (T1090),Protocol Tunneling (T1572),B,Masquerading (T1036) involves an adversary man...
2,Which MITRE ATT&CK technique involves an adver...,Process Injection,Obfuscated Files or Information,Data Obfuscation,Indicator Removal on Host,C,Obfuscating data can be used by threat actors ...
